# Функции в Pandas
Примеры из презентации

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'user_id': [1, 2, 3], 'clicks': [163, 130, 97], 'orders': [2, 4, 0]})
df

In [ ]:
df = pd.DataFrame({'user_id': [1, 2, 3], 'clicks': [163, 130, 97], 'orders': [2, 4, 0], 'calculated': [False, False, True]})
df[['user_id', 'clicks', 'orders', 'calculated']]

In [ ]:
def watcher(param):
    """Мне только посмотреть"""
    return param == 0

In [ ]:
df['watcher'] = df['orders'].apply(watcher)
df

Применяем метод apply к одному столбцу. Сейчас в переменную функции передаются значения одного столбца

In [ ]:
def conversion(row):
    """Подсчет конверсии переходов в покупки"""
    return row['orders'] / row['clicks']

### Как потестировать функцию со значениями одной строки

In [ ]:
row1 = pd.DataFrame({'clicks': [163], 
                     'orders': [2], 
                     'user_id': [1]})
conversion(row1)

Применяем метод apply к датафрейму. В переменную функции передаются строки целиком

In [ ]:
df = pd.DataFrame({'user_id': [1, 2, 3], 'clicks': [163, 130, 97], 'orders': [2, 4, 0]})
df[['user_id', 'clicks', 'orders']]

In [ ]:
df['conversion'] = df.apply(conversion, axis=1)

In [ ]:
df[['user_id', 'clicks', 'orders', 'conversion']]

### Как создавать столбцы с помощью функций

In [ ]:
df = pd.DataFrame({'user_id': [1, 2, 3]})

In [ ]:
df

In [ ]:
def metriks(row):
    """Возвращает новый столбец в зависимости от значения user_id"""
    user_id = row.user_id
    
    return pd.Series({'col_{}'.format(user_id): 1})

In [ ]:
df = df.apply(metriks, axis=1)
df

### Анализ рекламных кампаний

In [ ]:
import pandas as pd

In [ ]:
stats = pd.read_excel('ad_campaigns.xlsx')
stats.head()

In [ ]:
stats.columns = ['group', 'phrase', 'effect', 'ad_id', 'title', 'text', 'link']
stats.head()

### Lambda-функции
Хотим посчитать распределение количества слов в столбце с фразами

In [ ]:
phrase = 'МРТ на Менделеевской от 2000'

In [ ]:
len(phrase.split(' '))

In [ ]:
stats['word_count'] = stats['phrase'].apply(lambda word: len(word.split(' ')))
stats.head()

In [ ]:
# вариант с передачей всей строчки функции
# тут надо обязательно указать параметр axis = 1

stats['word_count'] = stats.apply(lambda x: len(x['phrase'].split(' ')), axis=1)
stats.head()

Простая визуализация

In [ ]:
%matplotlib inline

In [ ]:
stats['word_count'].hist()

In [ ]:
stats['word_count'].hist(bins=30)

### Упражнение
Поисковые запросы с каким количеством слов встречаются в наших данных чаще всего? Выведите топ-5

### Произвольные функции
В URL кампаний есть названия. С этим надо что-то делать

In [ ]:
# пример ссылки
url = stats.loc[0, 'link']
url

In [ ]:
from urllib import parse

In [ ]:
parse.urlsplit('https://ya.ru/news/sport?search=footbal#abc')

In [ ]:
parsed = parse.urlsplit(url)
parsed

In [ ]:
# можно конечно вручную
parsed.query.split('&')[2].split('=')[1]

In [ ]:
# как доставать значения

parsed.netloc

In [ ]:
params = parse.parse_qs(parsed.query)
params

In [ ]:
# вот и кампании

params['utm_campaign'][0]

In [ ]:
# зачем тут везде списки?

url_with_doubles = 'https://awesome-site.ru/?a=1&a=2&a=3'

parsed = parse.urlsplit(url_with_doubles)
parse.parse_qs(parsed.query)

In [ ]:
# оборачиваем все в функцию
# в качестве аргумента будет строка датафрейма

def campaign_name(row):
    """Получение названия кампании из ссылки внутри строки row"""

    parsed = parse.urlsplit(row['link'])
    params_dict = parse.parse_qs(parsed.query)

    return params_dict['utm_campaign'][0]

In [ ]:
# проверяем датафрейм
stats.head()

In [ ]:
stats['campaign'] = stats.apply(campaign_name, axis=1)
stats.head()

### Как передать в функцию несколько аргументов

In [ ]:
# как передать несколько аргументов

def power_up(row, n):
    """Возводит значение столбца effect в степень n"""
    return row['effect'] ** n

In [ ]:
stats['power_up'] = stats.apply(power_up, n=3, axis=1)
stats.head()

### Упражнение
В наших данных есть много объявлений с услугой МРТ (в столбце group есть слово 'мрт') круглосуточно (в тексте объявления text есть '24 часа'). Отфильтруйте строки, в которых НЕ упоминается МРТ, но прием идет круглосуточно. Сколько таких строк в датасете?

# Группировки

In [ ]:
df = pd.DataFrame({'order_id': [1, 2, 3, 4, 5], 'country': ['Россия', 'Китай', 'Китай', 'Россия', 'Россия'], 
                   'category': ['Электроника', 'Авто', 'Электроника', 'Авто', 'Авто'], 
                   'amount': [100, 80, 90, 140, 90]})
df

### Упражнение
Создайте датафрейм df_russia, в котором оставьте заказы из России. И аналогично df_china (заказы из Китая).

### Упражнение
Посчитайте для df_russia и df_china:
- среднюю стоимость заказа
- разницу между максимальной и минимальной стоимостью заказа

Объединим процесс разбиения на датафреймы

In [ ]:
def groupby_function(data):
    return data.amount.max() - data.amount.min()

In [ ]:
df.groupby('country').apply(groupby_function)

Вернемся к статистике рекламных кампаний

In [ ]:
# раньше использовали value_counts

stats['campaign'].value_counts().head()

In [ ]:
# более универсальный способ

stats.groupby('campaign').count().head()

In [ ]:
stats.groupby('campaign').count()[['group', 'effect']].head()

### Как вернуть столбец из индекса - метод reset_index()

In [ ]:
stats.groupby('campaign').count().reset_index().head()

### К группировке можно применять разные функции такие образом:

In [ ]:
obj = stats.groupby('campaign')

In [ ]:
obj.max().head()

In [ ]:
obj.mean().head()

### Упражнение
Для каждой кампании campaign найдите минимальное значение столбца effect и постройте рейтинг кампаний по этим значениям, начиная с самого низкого.

### Несколько функций в группировках

In [ ]:
# задаем несколько функций сразу

stats.groupby('campaign').agg(['min', 'max'])['effect'].head()

In [ ]:
# разные метрики для разных столбцов

results = stats.groupby('campaign').agg({'effect': ['min', 'max'], 'power_up': 'mean'})
results.head()

### Как обращаться к вложенным столбцам

In [ ]:
results['effect']['max'].head()

In [ ]:
# или даже так
stats.groupby('campaign').agg({'effect': ['min', 'max'], 'power_up': 'mean'})['effect']['max'].head()

In [ ]:
# группировка по нескольким столбцам

stats.groupby(['group', 'campaign']).count().head()

In [ ]:
# максимальное число объявлений в одной группе

stats.groupby(['group', 'campaign']).count().sort_values('phrase', ascending=False).head()

### Упражнение
Какая кампания имеет наименьшую сумму по столбцу effect? 

In [ ]:
#Домашнее задание

In [30]:
import pandas as pd

data = pd.read_csv('ml-latest-small/ratings.csv')
#data1 = data[['movieId', 'rating']].groupby(['movieId'])['rating'].mean().reset_index(name='mean')
data['class'] = ''
data.loc[data['rating'] <= 2, 'class'] = 'низкий рейтинг'
data.loc[data['rating'] <= 4, 'class'] = 'средний рейтинг'
data.loc[data['rating'] >= 4.5, 'class'] = 'высокий рейтинг'
print(data)


        userId  movieId  rating   timestamp            class
0            1        1     4.0   964982703  средний рейтинг
1            1        3     4.0   964981247  средний рейтинг
2            1        6     4.0   964982224  средний рейтинг
3            1       47     5.0   964983815  высокий рейтинг
4            1       50     5.0   964982931  высокий рейтинг
...        ...      ...     ...         ...              ...
100831     610   166534     4.0  1493848402  средний рейтинг
100832     610   168248     5.0  1493850091  высокий рейтинг
100833     610   168250     5.0  1494273047  высокий рейтинг
100834     610   168252     5.0  1493846352  высокий рейтинг
100835     610   170875     3.0  1493846415  средний рейтинг

[100836 rows x 5 columns]


In [51]:
regions = {'Центр': ['москва', 'тула', 'ярославль'],
           'Северо-Запад': ['петербург', 'псков', 'мурманск'],
           'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

data = pd.read_csv('keywords.csv')
data['region'] = 'undefined'
for k, v in regions.items():
    data.loc[data['keyword'].str.contains('|'.join(v)), 'region'] = k
data.to_csv('keywords1.csv')
print(data)

                       keyword     shows     region
0                           вк  64292779  undefined
1                одноклассники  63810309  undefined
2                        порно  41747114  undefined
3                         ютуб  39995567  undefined
4                    вконтакте  21014195  undefined
...                        ...       ...        ...
99995   эльдорадо старый оскол      3705  undefined
99996      frigate для firefox      3630  undefined
99997                   укрсиб      3630  undefined
99998  погода в ялте на неделю      3688  undefined
99999                     ггму      3630  undefined

[100000 rows x 3 columns]


In [50]:
years = [i for i in range(1950,2011)]

def production_year(s):
    for i in years:
        if str(i) in s:
            return i
    return 1900

data = pd.read_csv('ml-latest-small/ratings.csv')
films = pd.read_csv('ml-latest-small/movies.csv')
data1 = pd.merge(left=films, right=data, left_on='movieId', right_on='movieId')
data1['year'] = data1.apply(lambda x: production_year(x['title']), axis =  1)
data2 = data1[['year', 'rating']].groupby(['year'])['rating'].mean().reset_index(name='rating').sort_values('rating', ascending=False)
data2.to_csv('ml-latest-small/year_rating.csv')
print(data2)


    year    rating
8   1957  4.039535
5   1954  4.009191
13  1962  3.969466
3   1952  3.953125
23  1972  3.944293
..   ...       ...
51  2000  3.398922
56  2005  3.359976
43  1992  3.353555
48  1997  3.347241
47  1996  3.335329

[62 rows x 2 columns]
